In [ ]:
S2S and attention

统计机器翻译 SMT
- translate French -> English
- given French sentence x, find best English y
  - argmax_y P(y|x)
- 使用贝叶斯公式，将问题转为
  - argmax_y P(x|y)P(y)
  - 前者关注翻译，但需要大量的 f-e 翻译对，不现实。将问题拆解为 词与词 之间的翻译
  - 后者关注语言模型，即如何写出地道英语

神经机器翻译 NMT
- 使用一个神经网络
- 架构为 sequence-to-sequence，由两个 RNN 组成
- encoder RNN 会训练出句子的 encoding，作为 decoder 隐藏层的初始化状态
- decoder RNN 是一个 LM，从上面的 encoding 中生成目标句子
    - 注意，test time behavior，每次 decoder 的输出都会是下一个的输入
    - 注意，有 embedding 层，两
- NMT 是一个 Conditional Language Model（LM 的作用是根据 context 预测下一个词是什么）
    - 预测的概率 P(y|x) = P(y_1|x)P(y_2|y_1, x)P(y_3|y_2, y_1, x)...,P(y_T|y_1,...,t_T-1,x)
    - 注意，NMT 是直接预测 P(y|x)，而 SMT 将问题拆分为翻译模型和语言模型

NMT 对 SMT 的改进
- 更好的效果 - 更流畅、上下文关系更好、能使用词语之间的相似性
- 端到端训练
- 不需要特征工程，每种语言对都是相似的方法
- 但更难解释，更难控制规则

训练 NMT
- NMT 的训练是端到端的
- decoder 时，每次寻找每一步概率最大的词，即 greedy decoding
- beam search: greedy decoding 每一步的预测是无法更改的，容易出错，更好的办法是用 beam search 寻找多个选择
- beam search: decoder 的每一步，寻找最可能的 k 个最可能的 结果，k 一般为 5-10，虽然不能保证最佳的结果，但很高效

翻译效果衡量
- BLEU bilingual evaluation understudy
- 将机器翻译和人工翻译做相似度比较
- 相似度方式：n-gram 相似度，一般为 3-4 grams
- 过短的翻译会做惩罚
- BLEU 有用但不完美，很多好的翻译就因为 n-gram 重合度低而得分低
- SMT 需要几百个工程师很多年的时间，NMT 只需要几个工程师几个月时间

一些 trick
- make the encoder bidirectional
- obviously upgrade the RNN cell
- feed in the source backwards
- speed up the softmax computation
- pretrain on other stuff
- one network for all languages

NMT 还有一些问题
- out of vocab words
- 训练和测试样本的 domain mismatch
- 更长的 context 保持
- 低训练内容的翻译对
- 使用俚语、常识还是不够好
- 性别等会有 bias

attention 机制
- 对 source encoding 的时候，信息需要保存在最后的状态，造成信息瓶颈
- 在 decoder 阶段的每一步，关注 encoder 的每一步隐含状态的加权

attention 机制的过程
- encoder 的状态 h_1, ..., h_N
- decoder 当前的状态 s_t
- 将 s_t 和每一步 h_n 做点乘，使用 softmax 求出每一个的概率分布
- 将 encoder 的状态和概率分布加权得到 attention 向量
- 将 attention 和 s_t 组合后放回 s2s 的训练

attention 极大地提升了 NMT 的效果
- 使得 decoder 关注 encoder 特定的状态
- 解决了信息瓶颈的问题，因为它可以关注更往前的状态
- 减少了梯度消失的问题
- 增加了可读性 -> 从 attention 的分布可以看出 -> 网络自己在学习词语的映射关系！

S2S 的作用不只是 NMT
- 文本归纳
- 对话
- 文本解析
- 代码生成（自然语言 -> Python）